In [31]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [32]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
from dateutil.relativedelta import *
import time
import seaborn as sns

#SQL related - NEED TO DECIDE WHICH ONE I'LL BE USING AND DELETE THE REST
import sqlite3
import pandas.io.sql as pd_sql
# import psycopg2
# from sqlalchemy import create_engine

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 6000)

import warnings
warnings.filterwarnings('ignore')

In [33]:
def process_and_update(new_ads_df, existing_df, conn):
    """This function takes the new ads dataframe, the existing data from funda_ads table in the postgres database and the connection string.
    It goes through the records updating the is_duplicate, is_price_change and is_status_change columns where needed. 
    It check for and ignores complete duplicates. Eventually all the records that are not complete duplicates of data in the existing table
    are inserted into the database. 
    Does not return anything."""
    
    # Add a new column that will allow to keep track of which rows to keep and append to the existing table
    new_ads_df['is_to_keep'] = False
    
    # Create lists with only columns used to check for duplicates, status and price changes
    check_duplicates = ['property_id', 'title']
    check_complete_duplicate = ['property_id', 'title', 'Asking price (€)', 'Status']
    check_price_change = ['property_id', 'title', 'Status']
    check_status_change = ['property_id', 'title', 'Asking price (€)']

    
    # The code doesn't yet check if the same records wasn't in the new_ads df letting those records slip through the cracks
    
    for index, row in new_ads_df.iterrows():        
        if row[check_complete_duplicate].values.tolist() in existing_df[check_complete_duplicate].values.tolist():
            new_ads_df.at[index, 'is_to_keep'] = False
        elif row[check_price_change].values.tolist() in existing_df[check_price_change].values.tolist():
            new_ads_df.at[index, 'is_duplicate'] = True
            new_ads_df.at[index, 'is_price_change'] = True
            new_ads_df.at[index, 'is_to_keep'] = True
            # Assign a rank_same_record value
            row_id_title_pair = row[check_duplicates].values.tolist()
            new_data_df.at[index, 'rank_same_record'] = existing_df[check_duplicates].values.tolist().count(row_id_title_pair) + 1
            new_row = new_data_df.iloc[index]
            new_row.drop(columns=['listed_date_present', 'is_complete_duplicate', 'is_to_keep'], inplace=True)
            new_row.to_sql('funda_ads', con=conn, if_exists='append', index=False)
        elif row[check_status_change].values.tolist() in existing_df[check_status_change].values.tolist():
            new_ads_df.at[index, 'is_duplicate'] = True
            new_ads_df.at[index, 'is_status_change'] = True
            new_ads_df.at[index, 'is_to_keep'] = True
            row_id_title_pair = row[check_duplicates].values.tolist()
            new_data_df.at[index, 'rank_same_record'] = existing_df[check_duplicates].values.tolist().count(row_id_title_pair) + 1
            new_row = new_data_df.iloc[index]
            new_row.drop(columns=['listed_date_present', 'is_complete_duplicate', 'is_to_keep'], inplace=True)
            new_row.to_sql('funda_ads', con=conn, if_exists='append', index=False)
        else:
            new_ads_df.at[index, 'is_duplicate'] = False
            new_ads_df.at[index, 'is_status_change'] = False
            new_ads_df.at[index, 'is_to_keep'] = True
            row_id_title_pair = row[check_duplicates].values.tolist()
            new_data_df.at[index, 'rank_same_record'] = existing_df[check_duplicates].values.tolist().count(row_id_title_pair) + 1
            new_row = new_data_df.iloc[index]
            new_row.drop(columns=['listed_date_present', 'is_complete_duplicate', 'is_to_keep'], inplace=True)
            new_row.to_sql('funda_ads', con=conn, if_exists='append', index=False)
            
            
    final_new_df = new_ads_df[new_ads_df['is_to_keep'] == True] 
#     final_new_df.drop(columns=['listed_date_present', 'is_complete_duplicate', 'is_to_keep'], inplace=True)
    print(f'The database will be updated with additional {final_new_df.count()[0]} records.')
    
    # Now, update the database table with the new records
    final_new_df.to_sql('funda_ads', con=conn, if_exists='append', index=False)
    updated_df = pd.read_sql_query("""SELECT * FROM funda_ads""", con=conn)
    print(f'Now funda_ads table has {updated_df.count()[0]} records.') 
    
    
    # Finally, update the unique_funda_ads table with unique (latest) ads. 
    # This eliminates all the records where is_price_change and is_status_change is true
    updated_df['is_latest_record'] = False
    
    latest_ads_helper_df = updated_df.groupby('property_id')['rank_same_record'].max().reset_index()
    
    for index, row in updated_df.iterrows():
        if row[['property_id', 'rank_same_record']].values.tolist() in latest_ads_helper_df.values.tolist():
            updated_df.at[index, 'is_latest_record'] = True
    
    unique_records_updated_df = updated_df[updated_df['is_latest_record'] == True] 
    unique_records_updated_df.to_sql('unique_funda_ads', con=conn, if_exists='replace', index=False)
    print(f'unique_funda_ads table has been updated and now has {unique_records_updated_df.count()[0]} records.') 


def get_listed_date(value, scraped_date):
    """Converts a string in the 'Listed since' column into a listed_date value (in date format)"""
    today = dt.date.today()

    try:
        listed_date = dt.datetime.strptime(value, '%B %d, %Y').date()
    except:
        if scraped_date is np.NaN:
            listed_date = np.NaN    
        else:         
            if 'Today' in value:
                listed_date = scraped_date
            elif 'week' in value:
                weeks_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(weeks=weeks_listed)
            elif 'month' in value:
                months_listed = int(re.search('\d*', value).group())
                listed_date = scraped_date - relativedelta(months=months_listed)
            elif '6+' in value:
                # Not precise enough to tell, could be 7 months, could be 2 years
                listed_date = np.NaN
            else:
                listed_date = np.NaN
    return listed_date
    

def get_energy_label(value):
    """Takes 'Energy label' column and strips it from the words 'What does this mean?'
    Return just the label"""
    no_touch_list = ['Not required', 'Not available', np.NaN]
    if value not in no_touch_list:
        return value[0]
    else: 
        return np.NaN

def get_int(value):
    """Trims the price, area and other fields with numbers and converts them into int"""
    try:
        return re.sub('[€\sk.,m²m³v.o.n.permonthBeforeAfter]', '', value)
    except:
        return np.NaN
    

def get_rooms(value, room_type):
    """Retrieves the number of rooms specified by the type (room, bedroom, toilet, bathroom, etc.).
    If bedrooms are not specified and there is only 1 room - returns 0, if there is more than 1 room, but bedrooms not specified - returns NaN.
    Otherwise, returns the number of bedrooms"""
    try:
        value = value.lower()
    except:
        return np.NaN
    
    if room_type == 'room':
        try:
            return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'bedroom':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 0
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
    
    if room_type == 'toilet':
        try:
            if room_type not in value and int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d*\s[a-z]*\s?{room_type}', value).group().strip(f' separate {room_type}'))
        except:
            return np.NaN
        
    if room_type == 'bathroom':
        try:
            if room_type not in value and int(re.search(f'\d*\s[a-z]*\s?toilet', value).group().strip(f' separate {room_type}')) == 1:
                return 1
            elif room_type not in value:
                return np.NaN
            else:
                return int(re.search(f'\d* {room_type}', value).group().strip(f' {room_type}'))
        except:
            return np.NaN
        
    
def get_bath_flag(value):
    """Takes Bathroom facilities column and create a Bath_Flag column if a bathtub / bath is available in the property.
    Returns True or False"""
    try:
        if 'bath' in value.lower():
            return True
        else:
            return False
    except:
        return np.NaN
    
    
def get_facilities(value, facility_type):
    """Take the Bathroom facilities column and facility type (toilet, shower, bath, jacuzzi, steam cabin, etc.)
    and returns the number of specified facilities"""
    try:
        value = value.lower()
        facility_type = facility_type.lower()
    except:
        return np.NaN
    
    try:
        return int(re.search(f'\d* {facility_type}', value).group().strip(f' {facility_type}'))
    except:
        if facility_type in value:
            return 1
        else:
            return 0

In [34]:
# First things first, connect to the postgresql database
conn = sqlite3.connect('./Database/ams_market_watch.db')  # You can create a new database by changing the name within the quotes
# cursor = conn.cursor()

# Now pull in existing records from the database table called funda_ads
current_data_df = pd.read_sql_query("""SELECT * FROM funda_ads""", con=conn)
original_ad_count = len(current_data_df)

# Also load the data from new_adverts.pkl file 
with open('./Cellar/Archive/new_adverts_2020414.pkl', 'rb') as new_ads_pkl:
    new_data = pickle.load(new_ads_pkl)
# with open('./Cellar/new_adverts.pkl', 'rb') as new_ads_pkl:
#     new_data = pickle.load(new_ads_pkl)


# This is to create a pandas dataframe with the column sorting as in the dictionaries
column_list = []

for ad in new_data:
    for feat_name in list(ad.keys()):
        if feat_name not in column_list:
            column_list.append(feat_name)      
            

# Step 3 - Add "Listed_date_present", "listed_date" and "is_duplicate" columns
column_list.append('listed_date')
column_list.append('listed_date_present')
column_list.append('is_duplicate')
column_list.append('is_complete_duplicate')
column_list.append('rank_same_record')


# Initiate the dataframe with the desired columns
new_data_df = pd.DataFrame(columns=column_list)


# Iterate over the ads_list and replace "Listed since" field with a date (using get_listed_date function), where possible 
for ad in new_data:
    ad['listed_date'] = get_listed_date(ad['Listed since'], ad['scraped_date'])
    
    
check_complete_duplicate = ['property_id', 'title', 'Asking price', 'Status']
for ad in new_data:
    # Create a list with columns cords for checking if a record is a complete duplicate (cdpl)
    ad_check_cdpl_value = [ad.get(key) for key in check_complete_duplicate]
    
    # Check if new data dataframe does not have this record yet, and if so, add it. Otherwise, skip it
    if ad_check_cdpl_value not in new_data_df[check_complete_duplicate].values.tolist():
        new_data_df = new_data_df.append(ad, ignore_index=True)
    
    
# Convert columns to the right formats, clean up text values and make them numbers, etc.
new_data_df.drop(columns=['price'], inplace=True)
new_data_df['property_id'] = new_data_df['property_id'].apply(int)
new_data_df['listed_date'] = new_data_df.apply(lambda x: get_listed_date(x['Listed since'], x['scraped_date']), axis=1)
new_data_df['address'] = new_data_df['title']+', '+new_data_df['address']
new_data_df['Asking price'] = new_data_df['Asking price'].apply(get_int)
new_data_df['Asking price per m²'] = new_data_df['Asking price per m²'].apply(get_int)
new_data_df['VVE (Owners Association) contribution'] = new_data_df['VVE (Owners Association) contribution'].apply(get_int)
new_data_df['Year of construction'] = new_data_df['Year of construction'].apply(get_int)
new_data_df['Living area'] = new_data_df['Living area'].apply(get_int)
new_data_df['Exterior space attached to the building'] = new_data_df['Exterior space attached to the building'].apply(get_int)
new_data_df['Volume in cubic meters'] = new_data_df['Volume in cubic meters'].apply(get_int)
new_data_df['Rooms'] = new_data_df['Number of rooms'].apply(get_rooms, room_type='room')
new_data_df['Bedrooms'] = new_data_df['Number of rooms'].apply(get_rooms, room_type='bedroom')
new_data_df['Bathrooms'] = new_data_df['Number of bath rooms'].apply(get_rooms, room_type='bathroom')
new_data_df['Toilets'] = new_data_df['Number of bath rooms'].apply(get_rooms, room_type='toilet')
new_data_df['Has_Bathtub'] = new_data_df['Bathroom facilities'].apply(get_bath_flag)
new_data_df['Baths'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='bath')
new_data_df['Number of Toilets'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='toilet')
new_data_df['Showers'] = new_data_df['Bathroom facilities'].apply(get_facilities, facility_type='shower')
new_data_df['Energy label'] = new_data_df['Energy label'].apply(get_energy_label)
new_data_df['Provisional energy label'] = new_data_df['Provisional energy label'].apply(get_energy_label)

# Change the names of the columns to make it clearer / more accurate
new_data_df.rename(columns={'Asking price': 'Asking price (€)', 'Asking price per m²': 'Asking price per m² (€)', 
                       'VVE (Owners Association) contribution': 'VVE contribution (monthly) (€)',
                      'Living area': 'Living area (m²)', 'Volume in cubic meters': 'Volume (m³)'}, inplace=True)

# Finally, update the funda_ads postgres table with the new records after removing unnecessary duplicates
process_and_update(new_data_df, current_data_df, conn=conn)

OperationalError: table funda_ads has no column named 797

In [ ]:
# Finally, update the funda_ads postgres table with the new records after removing unnecessary duplicates
# process_and_update(new_data_df, current_data_df, conn=conn)

In [ ]:
# Now pull in existing records from the database table called funda_ads
funda_ads_df = pd.read_sql_query("""SELECT * FROM funda_ads""", con=conn)

print(f'Original number of records was {current_data_df.count()[0]}. The final number of records is {funda_ads_df.count()[0]}.')

In [ ]:
unique_funda_ads_df = pd.read_sql_query("""SELECT * FROM unique_funda_ads""", con=conn)
unique_funda_ads_df.head(1)

In [ ]:
funda_ads_df[funda_ads_df[['property_id', 'title']].duplicated()].sort_values(by='property_id', ascending=False)

In [ ]:
funda_ads_df[funda_ads_df['property_id'] == 87210295]      #87212811

In [ ]:
type(funda_ads_df.iloc[7551])